In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/allstate-purchase-prediction-challenge/train.csv.zip')
test = pd.read_csv('/kaggle/input/allstate-purchase-prediction-challenge/test_v2.csv.zip')


In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
pd.set_option('display.max_columns', None)
train

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
##########################  Psudo Code  ####################

#     time ===> extract hours
#     state, location ===> drop for now
#     car_value ===> create dummy variables
#     risk_factor ===> NaN predict with mean
#     all missing values predict with mean for now


In [ ]:
train_cl = train.copy()
# time extract hour
train_cl['time'] = train["time"].str[0:2].astype('int32')
# state, location
train_cl = train_cl.drop(['state','location'],axis=1)
# car value to dummy variables
train_cl = pd.get_dummies(train_cl, columns=['car_value'])
# columns contain missing values are
# car_value, 1531
# risk_factor, 240418
# C_previous, 18711
# duration_previous, 18711
# impute missing values with mean for now
train_cl = train_cl.fillna(train_cl.mean())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb

X = train_cl.drop(["record_type","customer_ID"],axis=1)
y = train_cl['record_type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=20)
X_train.shape

In [ ]:
help(lgb.LGBMClassifier)

In [ ]:
estimator = lgb.LGBMClassifier(learning_rate = 0.1, metric = 'l1', 
                        n_estimators = 10000)


param_grid = {
    'num_leaves ': [x for x in range(10,50,1)]
}
gridsearch = GridSearchCV(estimator, param_grid)

gridsearch.fit(X_train, y_train,
        eval_set = [(X_test, y_test)],
        eval_metric = ['auc', 'binary_logloss'],
        early_stopping_rounds = 5)

In [ ]:
print('Best parameters found by grid search are:', gridsearch.best_params_)